<H1> save_objects(image_path, output_dir, minimumPixelCount)</H1>

<h3>This is the first draft of the function that will take an image and convert it to subimages by trying to find contours using open CV.</h3>



In [72]:
import cv2
import os
import numpy as np

def preprocess_image(img):
    """
    Preprocesses the image by converting to grayscale and applying Gaussian blur.
    
    Parameters:
    img (numpy.ndarray): The input image.

    Returns:
    numpy.ndarray: The preprocessed image.
    """
    # Check if the image has an alpha channel
    if img.shape[2] == 3:
        # If not, convert the image to RGBA
        img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    return blurred, img

def apply_threshold(blurred):
    """
    Applies Otsu's thresholding to the image.

    Parameters:
    blurred (numpy.ndarray): The preprocessed image.

    Returns:
    numpy.ndarray: The thresholded image.
    """
    # Apply Otsu's thresholding
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return thresh




def find_contours(thresh):
    """
    Finds contours in the image.

    Parameters:
    thresh (numpy.ndarray): The thresholded image.

    Returns:
    list: A list of contours.
    ndarray: The hierarchical information.
    """
    # Find contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    
    return contours, hierarchy

def shrink_contour(contour, shrink_amount=3):
    # Compute the bounding box for the contour
    x, y, w, h = cv2.boundingRect(contour)

    # Shrink the bounding rectangle by a certain amount
    x += shrink_amount
    y += shrink_amount
    w -= 2 * shrink_amount
    h -= 2 * shrink_amount

    return x, y, w, h

def feather_mask(mask, feather_amount=3):
    # Blur the mask to create a "feather" effect
    return cv2.GaussianBlur(mask, (feather_amount, feather_amount), 0)

def extract_objects(img, contours, hierarchy, minimumPixelCount):
    """
    Extracts objects from the image based on contours.

    Parameters:
    img (numpy.ndarray): The input image.
    contours (list): A list of contours.
    hierarchy (ndarray): The hierarchical information.
    minimumPixelCount (int): The minimum number of pixels an object should have.

    Returns:
    list: A list of extracted objects.
    """
    objects = []
    
    # Iterate over the contours
    for i, contour in enumerate(contours):
        # Calculate actual contour area
        contourArea = cv2.contourArea(contour)
        
        # Check if the contour area is at least as large as minimumPixelCount
        if contourArea >= minimumPixelCount:
          
            # Shrink the contour
            x, y, w, h = shrink_contour(contour)

            # Extract the object and save it to a file
            obj = img[y:y+h, x:x+w].copy()

            # Create a mask and draw the contour on it
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.drawContours(mask, [contour], -1, (255), thickness=cv2.FILLED, offset=(-x, -y))

            # Feather the mask
            mask = feather_mask(mask)

            # Apply the mask to the alpha channel of the object, making the area outside the contour transparent
            obj[..., 3] = mask
            
            # Get the hierarchy level of the current contour
            hierarchy_level = hierarchy[0, i, 3]
            
            objects.append(obj)
    
    return objects

def process_image(image_path, minimumPixelCount):
    # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    print(f"Processing at image_path: {image_path}")
    
    # Preprocess the image
    blurred, img = preprocess_image(img)
    
    # Apply threshold
    thresh = apply_threshold(blurred)
    
    # Find contours
    contours, hierarchy = find_contours(thresh)
    
    # Extract objects
    objects = extract_objects(img, contours, hierarchy, minimumPixelCount)
    print(f".. Found {len(objects)}")    
    return objects  # Return only the list of objects

def process_images_in_folder(input_dir, output_dir, min_pixel_count=50*50):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # List all files in the directory
    files = os.listdir(input_dir)

    # Loop through each file
    for file in files:
        # Check if the file is a PNG image
        if file.endswith(".png"):
            # Construct the full image path
            image_path = os.path.join(input_dir, file)

            # Process the image
            processed_images = process_image(image_path, min_pixel_count)
            print(f"Received {len(processed_images)}")
            # Save each processed image
            for i, image in enumerate(processed_images):
                # Construct the output file path
                output_path = os.path.join(output_dir, f'{os.path.splitext(file)[0]}_{i}.png')

                print(f'.. Attempting to write "{output_path}" at i={i}.')
                
                # Save the image
                cv2.imwrite(output_path, image)
                
                
# input folder = "./inputs/" <- need to exclude non .png files
# output folder = "./outputs/"

input_dir ="inputs"
output_dir = "output"

process_images_in_folder(input_dir, output_dir, min_pixel_count=50*50)


Processing at image_path: inputs\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_3aedb7a2-27cf-400d-a4ec-4f2852fa6d4a.png
.. Found 45
Received 45
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_3aedb7a2-27cf-400d-a4ec-4f2852fa6d4a_0.png" at i=0.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_3aedb7a2-27cf-400d-a4ec-4f2852fa6d4a_1.png" at i=1.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_3aedb7a2-27cf-400d-a4ec-4f2852fa6d4a_2.png" at i=2.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_3aedb7a2-27cf-400d-a4ec-4f2852fa6d4a_3.png" at i=3.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_3aedb7a2-27cf-400d-a4ec-4f2852fa6d4a_4.png" at i=4.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_3aedb7a2

.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_83484fa1-d512-4aac-8ede-db02d07f77f7_5.png" at i=5.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_83484fa1-d512-4aac-8ede-db02d07f77f7_6.png" at i=6.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_83484fa1-d512-4aac-8ede-db02d07f77f7_7.png" at i=7.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_83484fa1-d512-4aac-8ede-db02d07f77f7_8.png" at i=8.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_83484fa1-d512-4aac-8ede-db02d07f77f7_9.png" at i=9.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_83484fa1-d512-4aac-8ede-db02d07f77f7_10.png" at i=10.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_83484fa1-d512-4aac-8e

.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_9bd663fa-c439-46c1-a426-5a12e090a724_14.png" at i=14.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_9bd663fa-c439-46c1-a426-5a12e090a724_15.png" at i=15.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_9bd663fa-c439-46c1-a426-5a12e090a724_16.png" at i=16.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_9bd663fa-c439-46c1-a426-5a12e090a724_17.png" at i=17.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_9bd663fa-c439-46c1-a426-5a12e090a724_18.png" at i=18.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_9bd663fa-c439-46c1-a426-5a12e090a724_19.png" at i=19.
.. Attempting to write "output\CarpetShark_game_assets_jungle_theme_digital_matte_cartoonish_b_9bd663fa-c4